## LLaMA2 chat with SQL

Open source, local LLMs are great to consider for any application that demands data privacy.

SQL is one good example. 

This cookbook shows how to perform text-to-SQL using various local versions of LLaMA2 run locally.

## Packages

In [2]:
#! pip install langchain replicate

## LLM

There are a few ways to access LLaMA2.

To run locally, we use Ollama.ai. 

See [here](https://python.langchain.com/docs/integrations/chat/ollama) for details on installation and setup.

Also, see [here](https://python.langchain.com/docs/guides/local_llms) for our full guide on local LLMs.
 
To use an external API, which is not private, we can use Replicate.

In [3]:
#load API key
import os
def read_api_key(file_path):
    with open(file_path, 'r') as file:
        return file.read().strip()

REPLICATE_API_TOKE = read_api_key('../API_Key/REPLICATE_API_TOKEN.txt')
OPENAI_API_KE = read_api_key('../API_Key/OPENAI_API_KEY.txt')

os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKE
os.environ["OPENAI_API_KEY"] = OPENAI_API_KE


In [4]:
# Local
from langchain.chat_models import ChatOllama

llama2_chat = ChatOllama(model="llama2:13b-chat")
llama2_code = ChatOllama(model="codellama:7b-instruct")

# API
from langchain.llms import Replicate

# REPLICATE_API_TOKEN = getpass()
# os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN
replicate_id = "meta/llama-2-13b-chat:f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d"
llama2_chat_replicate = Replicate(
    model=replicate_id, model_kwargs={"temperature": 0.01, "max_length": 500, "top_p": 1}
)

In [5]:
# Simply set the LLM we want to use
llm = llama2_chat

## DB

Connect to a MySQL DB.

To create this particular DB, you can use the code and follow the steps shown [here](https://github.com/facebookresearch/llama-recipes/blob/main/demo_apps/StructuredLlama.ipynb).

In [6]:
from typing import Any, Dict, List, Tuple, Union

import pymysql

KV = Dict[str, Any]
Query = Tuple[str, List]

class DB:
    def __init__(self, host: str, port: int, user: str, password: str, database: str):
        conn = pymysql.connect(
            host=host,
            port=port,
            user=user,
            password=password,
            database=database,
            cursorclass=pymysql.cursors.DictCursor,
        )
        self.conn = conn
        self.database = database

    def get_cursor(self):
        return self.conn.cursor()
    
    def execute_query(self, query: str, args: List, ret_result: bool) -> Union[List[KV], int]:

        cur = self.get_cursor()
        count = cur.execute(query, args=args)
        if ret_result:
            return cur.fetchall()
        else:
            return count
        
    # def get_schema(self) -> List[KV]:
    #     query = "SELECT * FROM information_schema.tables"
    #     return self.execute_query(query, [], ret_result=True)
    def get_table_info(self, table: str) -> List[KV]:
        query = "DESCRIBE {}".format(table)
        return self.execute_query(query, [], ret_result=True)

In [7]:
from langchain.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///nba_roster.db", sample_rows_in_table_info=0)

# db = DB(
#     host="localhost",
#     port=3306,
#     user="root",
#     password="luweitao",
#     database="db_book_hw1",
# )
print(db.get_table_info())


def get_schema(_):
    return db.get_table_info()


def run_query(query):
    return db.run(query)


CREATE TABLE nba_roster (
	"Team" TEXT, 
	"NAME" TEXT, 
	"Jersey" TEXT, 
	"POS" TEXT, 
	"AGE" INTEGER, 
	"HT" TEXT, 
	"WT" TEXT, 
	"COLLEGE" TEXT, 
	"SALARY" TEXT
)


## Query a SQL DB 

Follow the runnables workflow [here](https://python.langchain.com/docs/expression_language/cookbook/sql_db).

In [8]:
# Prompt
from langchain.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Given an input question, convert it to a SQL query. No pre-amble."),
        ("human", template),
    ]
)

# Chain to query
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

sql_response.invoke({"question": "What team is Klay Thompson on?"})

'\nSELECT "Team"\nFROM nba_roster\nWHERE "NAME" = \'Klay Thompson\';'

We can review the results:

* [LangSmith trace](https://smith.langchain.com/public/afa56a06-b4e2-469a-a60f-c1746e75e42b/r) LLaMA2-13 Replicate API
* [LangSmith trace](https://smith.langchain.com/public/2d4ecc72-6b8f-4523-8f0b-ea95c6b54a1d/r) LLaMA2-13 local 


In [9]:
# Chain to answer
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input question and SQL response, convert it to a natural language answer. No pre-amble.",
        ),
        ("human", template),
    ]
)

full_chain = (
    RunnablePassthrough.assign(query=sql_response)
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response
    | llm
)

full_chain.invoke({"question": "How many unique teams are there?"})

AIMessage(content='Natural language answer: There are 30 unique teams in the NBA.')

We can review the results:

* [LangSmith trace](https://smith.langchain.com/public/10420721-746a-4806-8ecf-d6dc6399d739/r) LLaMA2-13 Replicate API
* [LangSmith trace](https://smith.langchain.com/public/5265ebab-0a22-4f37-936b-3300f2dfa1c1/r) LLaMA2-13 local 

## Chat with a SQL DB 

Next, we can add memory.

In [10]:
# Prompt
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

template = """Given an input question, convert it to a SQL query. No pre-amble. Based on the table schema below, write a SQL query that would answer the user's question:
{schema}
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True)

# Chain to query with memory
from langchain_core.runnables import RunnableLambda

sql_chain = (
    RunnablePassthrough.assign(
        schema=get_schema,
        history=RunnableLambda(lambda x: memory.load_memory_variables(x)["history"]),
    )
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)


def save(input_output):
    output = {"output": input_output.pop("output")}
    memory.save_context(input_output, output)
    return output["output"]


sql_response_memory = RunnablePassthrough.assign(output=sql_chain) | save
sql_response_memory.invoke({"question": "What team is Klay Thompson on?"})

'\nSELECT "Team" FROM nba_roster WHERE "NAME" = \'Klay Thompson\';'

In [11]:
# Chain to answer
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input question and SQL response, convert it to a natural language answer. No pre-amble.",
        ),
        ("human", template),
    ]
)

full_chain = (
    RunnablePassthrough.assign(query=sql_response_memory)
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response
    | llm
)

full_chain.invoke({"question": "What is his salary?"})

AIMessage(content="Natural language answer:\nKlay Thompson's salary is $43,219,440.")

In [12]:
variable = "Hello, world!"
print(variable)
# Bellman-Ford Algorithm

def bellman_ford(graph, source):
    # Step 1: Initialize distances and predecessors
    distances = {node: float('inf') for node in graph}
    distances[source] = 0
    predecessors = {node: None for node in graph}

    # Step 2: Relax edges repeatedly
    for _ in range(len(graph) - 1):
        for node in graph:
            for neighbor in graph[node]:
                # Calculate the new distance
                new_distance = distances[node] + graph[node][neighbor]
                # Update the distance and predecessor if a shorter path is found
                if new_distance < distances[neighbor]:
                    distances[neighbor] = new_distance
                    predecessors[neighbor] = node

    # Step 3: Check for negative cycles
    for node in graph:
        for neighbor in graph[node]:
            # If a shorter path is found, there is a negative cycle
            if distances[node] + graph[node][neighbor] < distances[neighbor]:
                raise ValueError("Negative cycle detected")

    return distances, predecessors

# Example usage
graph = {
    'A': {'B': 5, 'C': 2},
    'B': {'D': 4},
    'C': {'B': 1, 'D': 3},
    'D': {'A': 1, 'B': 2}
}

source_node = 'D'
distances, predecessors = bellman_ford(graph, source_node)
 
# Print the distances and predecessors
for node in graph:
    print(f"Distance from {source_node} to {node}: {distances[node]}")
    print(f"Predecessor of {node}: {predecessors[node]}")


Hello, world!
Distance from D to A: 1
Predecessor of A: D
Distance from D to B: 2
Predecessor of B: D
Distance from D to C: 3
Predecessor of C: A
Distance from D to D: 0
Predecessor of D: None


Here is the [trace](https://smith.langchain.com/public/54794d18-2337-4ce2-8b9f-3d8a2df89e51/r).